# Pytorch实现神经概率语言模型

## 赵振东

### 本代码主要实现神经概率语言模型，使用了10000句微博用户文本数据。效果可能不会很好，主要是做个小实验演示一下，数据量有点小。

#### 先看一下数据长什么样子：

小马 也 疯狂 ------ 地位 之 争 。<br>
那些 年 ， 我们 一起 偷看 过 的 电视 。 「 暴 走 漫画 」<br>
北京 的 小 纯洁们 ， 周日见 。 # 硬汉 摆 拍 清纯 照 #<br>
要是 这 一 年 哭泣 的 理由 不 再 是 难过 而 是 感动 会 多么 好<br>
对于 国内 动漫 画作者 引用 工笔 素材 的 一些 个人 意见 。<br>
猫咪 保镖 最 赞 了 ！ 你们 看懂 了 吗 ？ ！ （ 来自 ： 9gag ）<br>
莫愁 和 所有 人 开 了 一 个 玩笑 —— 其实 ， 她 是 会 正常 唱歌 的 … …<br>
你 见 过 皮卡丘 喝水 的 样子 吗 ？<br>
如果 有 个 人 能 让 你 忘掉 过去 ， 那TA 很 可能 就是 你 的 未来 。<br>
我 在 北京 ， 24 岁 ， 想 去 马尔代夫 ， 一 个 人 。<br>
哥 你 还 跳 不 跳楼 了 ？ 我们 要 下班 啊 ！<br>
龙 生 龙 ， 凤 生 凤 ， 是 个 喵咪 它 就萌 。<br>
从 胚胎 期 开始 的 面部 特征 演变 过程<br>
本 届 轮值 主席 王石致 开幕词 。 讲 60 岁 上 哈佛 。<br>
非常 不 喜欢 北京 现在 的 天气 … … 非常 … …<br>
我 第一 次 坐 飞机 是 进 安达 信 的 入职 培训 ， 在 深圳 。 你们 哪 ？<br>
人生 如 戏 ， 全 靠 演技 。  小 受 吓坏 了 。<br>
为什么 这 世上 会 有 人 以 刁难 他人 为乐 呢 ？<br>
算了 算了 ， 我 看出来 了 ， 你们 都 想 看 男人 ！ 上 张 美 男图 。<br>
偷拍 时 被 喵星 人 发现 了 。 ！ \ 3 /<br>
看看 你 的 名字 在 古代 是 什么 职业 ， 太 让 人 崩溃 了 。<br>
居然 把 Windows Phone 8 写成 Win8 了 …… 要 严谨 啊<br>
百合 真 香 啊 ！ 送给 回家 路上 的 人们 明天 后天 就 不行了 ！<br>
刘翔 预算 摔倒 ！ 无缘 半决赛 ！ 主持人 哭了 。<br>
谁 身边 没 几 个 能办 大 事 的 朋友 ？<br>
一 位 妈妈 将 她 四 岁 儿子 的 涂鸦 做成 了 真实 的 毛绒 玩具<br>
竟然 下雪 了 ， 不 喜欢 冬天 ， 天气 何时 才 变暖 啊 。<br>
困扰 我 多 年 的 问题 终于 得 解了 。<br>
大学生 一定 要 看 的 一 分钟 ， 它 能 让 你 奋斗 一辈子 alink<br>

#### 上面是30句样本文本，现在我们开始编写代码实现这个语言模型吧

In [11]:
from nltk.util import ngrams
from numpy.random import shuffle
import torch.nn as nn
from torch.autograd import Variable
import torch
import numpy as np
import sys
import torch.optim as optim
from visdom import Visdom
from collections import Counter

class deta_loader(object):
    def __init__(self, N_gram=3,data_file=""):
        super(deta_loader,self).__init__()
        """
        数据加载函数，实现文本数据的加载和处理
        :param N_gram代表设置大小，即取前N_gram-1个词做为上下文
        :param data_file,是传入的文本文件，要求每一行一个句子，然后用空格分好词
        """
        self.N_gram=N_gram
        self.data_file=data_file
        self.words=[]
        self.sents=[]
        self.ids=[]
        self.red_file()
        self.word2idx=dict(zip(self.words,range(len(self.words))))
        self.idx2word=dict(zip(range(len(self.words)),self.words))
        self.sent2ids()
        self.vocab_size=len(self.word2idx)
        self.samples=[]
        self.gen_ngram()
        
    def red_file(self):
        """
        读文件函数，将文本文件读入数据
        """
        with open(self.data_file) as f:
            for line in f.readlines():
                #对每一行分开空格为list
                line=line.strip().split()
                self.words.extend(line)
                self.sents.append(line)
                
        self.all_words=self.words
        self.words=list(set(self.words))
                
    def sent2ids(self):
        """
        将文本数据转换为id数字化
        """
        for sent in self.sents:
            self.ids.append([self.word2idx[w] for w in sent])
                
    def gen_ngram(self):
        """
        生成N-gram样本函数
        """
        for sent in self.ids:
            b = ngrams(sent,self.N_gram)
            self.samples.extend([(list(item)[:self.N_gram-1],list(item)[-1]) for item in b])
        shuffle(self.samples)
        
    def get_most_common(self,k):
        c=Counter(self.all_words)
        return c.most_common(k)
        

In [12]:
class nlm(nn.Module):
    """语言模型网络类"""
    def __init__(self, emb_dim, vocab_size):
        super(nlm, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        self.proj=nn.Linear(emb_dim, vocab_size)
        self.softmax = nn.Softmax()
        
    def forward(self, inp):
        xw=self.word_emb(inp).sum(1)
        out=self.proj(xw).exp()
        self.softmax(out)
        return out
    

In [6]:

if __name__ == '__main__':
    
    viz = Visdom()
    line = viz.line(np.arange(2))
    
    epochs=20
    dl=deta_loader(data_file="weibo_data.txt")
    model=nlm(emb_dim=128,vocab_size=dl.vocab_size).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer=optim.Adam(model.parameters(),lr=0.0005)
    
    loss_list=[]
    step_p=[]
    loss_p=[]
    
    for epoch in range(epochs):
        for step,sample in  enumerate(dl.samples):
            inp,tag=sample
            inp=Variable(torch.from_numpy(np.array(inp))).view(1,-1).cuda()
            tag=Variable(torch.LongTensor(torch.from_numpy(np.array([tag])))).cuda()
            output=model(inp)
            loss = criterion(output,tag)
            optimizer.zero_grad()
            loss.backward()
            loss_list.append(loss.data[0])
            # 剪裁参数梯度
            optimizer.step()
            #每1000步打印loss
            if step%1000==0 :
                loss_p.append(np.mean(loss_list))
                step_p.append(step+epoch*len(dl.samples))
                viz.line(
                     X=np.array(step_p),
                     Y=np.array(loss_p),
                     win=line,
                     opts=dict(legend=["Train_mean_loss"]))
                
                loss_list=[]
            
            
            


/home/lab713/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
/home/lab713/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


KeyboardInterrupt: 

In [7]:
print(dl.get_most_common(500))

[('，', 7362), ('。', 6114), ('的', 5624), ('！', 3002), ('了', 2165), ('是', 1954), ('一', 1870), ('你', 1788), ('我', 1652), ('？', 1646), ('不', 1285), ('#', 1037), ('有', 1001), ('这', 937), ('个', 905), ('…', 876), ('在', 841), ('人', 748), ('转', 736), ('）', 700), ('（', 691), ('alink', 684), ('就', 654), ('：', 617), ('都', 605), ('【', 538), ('】', 538), ('会', 444), ('啊', 435), ('要', 421), ('好', 405), ('吗', 392), ('和', 383), ('能', 377), ('最', 356), ('很', 354), ('看', 347), ('什么', 337), ('这个', 333), ('也', 320), ('」', 317), ('「', 315), ('“', 311), ('大', 305), ('”', 304), ('太', 295), ('今天', 288), ('图', 284), ('想', 283), ('还', 277), ('过', 275), ('没', 267), ('吧', 266), ('我们', 260), ('吃', 252), ('自己', 251), ('来', 247), ('上', 246), ('说', 245), ('这样', 240), ('天', 237), ('中', 235), ('谁', 228), ('《', 227), ('》', 227), ('小', 224), ('去', 221), ('中国', 220), ('给', 219), ('喜欢', 215), ('大家', 214), ('、', 214), ('让', 209), ('北京', 208), ('——', 208), ('被', 208), ('种', 206), ('只', 203), ('爱', 200), ('没有', 197), ('又', 195)

In [71]:
def get_vec(w,embedding=model.word_emb,word2idx=dl.word2idx):
    w= torch.tensor([word2idx[w]]).cuda()
    w_vec=embedding(w).data.cpu().numpy()
    return w_vec

In [72]:
print(get_vec("时间"))

[[-4.4721228e-01  4.8448700e-01  1.5144777e+00  9.5147753e-01
   1.1991516e+00  2.3802574e-01 -3.2100305e-01 -1.3965905e-01
  -5.8980620e-01 -1.3014680e+00  1.2092247e+00  3.9012423e-01
  -2.0070937e-01 -6.4108288e-01  1.0200247e+00 -1.3535341e+00
  -1.4325051e+00  2.9552910e-01 -1.2998731e-01  8.7058049e-01
  -8.0749679e-01  1.1529236e+00  1.2957534e+00 -9.7147238e-01
   1.2506478e+00 -1.7023355e+00 -1.2845010e+00  4.6664897e-01
  -2.7361210e-02  1.5077260e+00 -1.0384908e+00  6.1781924e-02
   2.9631966e-01 -2.1747442e-01  8.3622068e-02  1.6053553e-01
   3.5777304e-01 -9.4086355e-01  8.7050891e-01  2.2529464e+00
   4.8131478e-01 -1.6052644e-01 -1.2991546e+00 -7.6134598e-01
   2.8889695e-01  1.1788763e+00  4.1251451e-01 -1.5097270e+00
   2.0492241e-02 -5.1674538e-04 -2.5935450e-01  9.6186481e-02
   1.0805404e+00  5.0584618e-02  8.3632082e-01 -4.2637479e-02
   1.4517438e+00 -1.8101935e+00 -2.9617688e-01  1.3959101e+00
  -1.2961789e+00 -1.6156006e+00  4.0722337e-01  1.6145271e-01
  -2.014

In [73]:
def sim(w1,w2,embedding=model.word_emb,word2idx=dl.word2idx):
    """
    利用训练好的模型计算cos相似度
    """
    w1= torch.tensor([word2idx[w1]]).cuda()
    w2= torch.tensor([word2idx[w2]]).cuda()
    w1_vec=embedding(w1).data.cpu().numpy()
    w2_vec=embedding(w2).data.cpu().numpy()
    
    num = float(np.dot(w1_vec,w2_vec.T)) 
    denom = np.linalg.norm(w1_vec) * np.linalg.norm(w2_vec)
    cos = num / denom #余弦值
    sim = 0.5 + 0.5 * cos #归一化   
    print(sim)
    


In [74]:
sim('美国','非常')
sim('美国','香港')
sim('美国','北京')
sim('美国','日本')

0.4606666786294646
0.5187198135575075
0.5678678651780635
0.595745311236103
